In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 产生数据
file = 'data/base1_fill_mean_normalized.csv'
data = pd.read_csv(file)
data

,icustay_id,starttime,endtime,rate_after,subject_id,is_male,age,ethnicity,weight,mingcs,...,Creatinine,WBC,MAP,Respiratory_Rate,Heart_Rate,Temperature_C,SPO2,PEEP,time_diff,rate_before
0,200075,2159/9/23 1:40,2159/9/23 2:25,0.130022,67800,0,0.894291,WHITE,58.1,0.916667,...,0.059701,0.064885,0.236486,0.377049,0.508876,0.675000,0.948276,0.000000,0.009428,0.000000
1,200075,2159/9/23 2:25,2159/9/23 2:46,0.149992,67800,0,0.894291,WHITE,58.1,0.916667,...,0.059701,0.064885,0.182432,0.393443,0.455621,0.675000,0.948276,0.000000,0.004285,0.130022
2,200075,2159/9/23 2:46,2159/9/23 3:50,0.179990,67800,0,0.894291,WHITE,58.1,0.916667,...,0.059701,0.064885,0.179054,0.393443,0.449704,0.675000,0.948276,0.000000,0.013499,0.149992
3,200075,2159/9/23 3:50,2159/9/23 4:59,0.199955,67800,0,0.894291,WHITE,58.1,0.916667,...,0.059701,0.064885,0.250000,0.426230,0.431953,0.675000,1.000000,0.000000,0.014570,0.179990
4,200075,2159/9/23 4:59,2159/9/23 5:12,0.239853,67800,0,0.894291,WHITE,58.1,0.916667,...,0.059701,0.064885,0.182432,0.311475,0.449704,0.675000,1.000000,0.000000,0.002571,0.199955
5,200075,2159/9/23 5:12,2159/9/23 6:03,0.269512,67800,0,0.894291,WHITE,58.1,0.916667,...,0.059701,0.064885,0.226351,0.262295,0.437870,0.675000,0.965517,0.208333,0.010714,0.239853
6,200075,2159/9/23 6:03,2159/9/23 6:37,0.229478,67800,0,0.894291,WHITE,58.1,0.916667,...,0.059701,0.064885,0.209459,0.295082,0.408284,0.675000,0.982759,0.208333,0.007071,0.269512
7,200075,2159/9/23 6:37,2159/9/23 8:04,0.209338,67800,0,0.894291,WHITE,58.1,0.916667,...,0.059701,0.064885,0.195946,0.311475,0.396450,0.675000,1.000000,0.208333,0.018427,0.229478
8,200075,2159/9/23 8:04,2159/9/23 8:20,0.179521,67800,0,0.894291,WHITE,58.1,0.916667,...,0.059701,0.064885,0.189189,0.295082,0.366864,0.670000,1.000000,0.208333,0.003214,0.209338
9,200075,2159/9/23 8:20,2159/9/23 9:16,0.149679,67800,0,0.894291,WHITE,58.1,0.916667,...,0.059701,0.064885,0.189189,0.295082,0.366864,0.670000,1.000000,0.208333,0.011785,0.179521


In [3]:
'''
DATA_NUM 一段时间序列的长度
'''
DATA_NUM = 10
icu_id = 0
end_time = '0000'
seg = []
X_tmp = []
# 参与聚类的变量，不确定age、weight、is_male、体温等属性是否需要
variables = ['rate_before', 'age', 'Bilirubin', 'PaO2', 'Creatinine', 'WBC', 'MAP',
             'Respiratory_Rate', 'Heart_Rate', 'SPO2', 'Temperature_C', 'PEEP', 'mingcs', 'is_male']
# variables = ['time_diff_nor', 'rate_after'] + ['x' + str(i) for i in range(5)]

In [4]:
def if_series(start_id):
    series = []
    # 记录当前icustay_id和结束时间
    icu_id = data.icustay_id[start_id]
    cur_end_time = data.endtime[start_id]
    # 加入这一条数据
    series.extend([data[v][start_id] for v in variables])
    # 判断下面DATA_NUM - 1条数据
    for i in range(start_id + 1, start_id + DATA_NUM):
        # 如果新的数据的icustay_id不等于上一条的id或者时间连不上，返回None
        if data.icustay_id[i] != icu_id or data.starttime[i] != cur_end_time:
            return None
        cur_end_time = data.endtime[i]

        series.extend([data[v][i] for v in variables])
    series.append(data['rate_after'][start_id + DATA_NUM - 1])
    return series

In [5]:
for i in range(len(data) - DATA_NUM + 1):
    # 重新记录一个Seg
    if icu_id != data.icustay_id[i]:
        icu_id = data.icustay_id[i]

    series = if_series(i)
    if series is not None:
        X_tmp.append(series)

In [6]:
X_tmp = np.array(X_tmp)
X_tmp.shape

(4119, 141)

In [7]:
# 将数据与要预测的结果分隔开
X_tmp = np.array(X_tmp)
np.random.shuffle(X_tmp)
y = X_tmp[:, -1]
y = y.reshape((y.shape[0], 1))
X_tmp = X_tmp[:, 0: DATA_NUM * len(variables)]
X = X_tmp.reshape((X_tmp.shape[0], DATA_NUM, len(variables)))
y.shape, X.shape

((4119, 1), (4119, 10, 14))

In [8]:
X[0]

array([[0.10041256, 0.43893697, 0.02513966, 0.21169644, 0.03731343,
        0.02841391, 0.25337838, 0.31147541, 0.65088757, 0.96551724,
        0.83      , 0.        , 1.        , 0.        ],
       [0.12053981, 0.43893697, 0.02513966, 0.21169644, 0.03731343,
        0.02841391, 0.20945946, 0.36065574, 0.59171598, 0.94827586,
        0.83      , 0.        , 1.        , 0.        ],
       [0.14071245, 0.43893697, 0.02513966, 0.21169644, 0.03731343,
        0.02841391, 0.20945946, 0.36065574, 0.59171598, 0.94827586,
        0.83      , 0.        , 1.        , 0.        ],
       [0.16074919, 0.43893697, 0.02513966, 0.21169644, 0.03731343,
        0.02841391, 0.24324324, 0.50819672, 0.62130178, 0.93103448,
        0.84      , 0.        , 1.        , 0.        ],
       [0.20082988, 0.43893697, 0.02513966, 0.21169644, 0.03731343,
        0.02841391, 0.23986486, 0.47540984, 0.63313609, 0.87931034,
        0.85      , 0.        , 1.        , 0.        ],
       [0.18057814, 0.43893697, 0.0

In [9]:
# 将X的后两个维度转置，使得第二维表示数据维度，第三维表示时间维度（Timesteps）
# 也可用X = X.transpose((0, 2, 1))
X = X.swapaxes(1, 2)
X.shape, X[0]

((4119, 14, 10),
 array([[0.10041256, 0.12053981, 0.14071245, 0.16074919, 0.20082988,
         0.18057814, 0.1404997 , 0.1204283 , 0.10035692, 0.08028553],
        [0.43893697, 0.43893697, 0.43893697, 0.43893697, 0.43893697,
         0.43893697, 0.43893697, 0.43893697, 0.43893697, 0.43893697],
        [0.02513966, 0.02513966, 0.02513966, 0.02513966, 0.02513966,
         0.02513966, 0.02513966, 0.02513966, 0.02513966, 0.02513966],
        [0.21169644, 0.21169644, 0.21169644, 0.21169644, 0.21169644,
         0.21169644, 0.21169644, 0.21169644, 0.21169644, 0.21169644],
        [0.03731343, 0.03731343, 0.03731343, 0.03731343, 0.03731343,
         0.03731343, 0.02985075, 0.02985075, 0.02985075, 0.02985075],
        [0.02841391, 0.02841391, 0.02841391, 0.02841391, 0.02841391,
         0.02841391, 0.01887193, 0.01887193, 0.01887193, 0.01887193],
        [0.25337838, 0.20945946, 0.20945946, 0.24324324, 0.23986486,
         0.20945946, 0.22972973, 0.22972973, 0.20945946, 0.20945946],
        [0

In [14]:
# 划分训练集、验证集、测试集
VALIDATIGN_END = int(0.8 * X.shape[0])
TRAINING_END = int(0.8 * VALIDATIGN_END)

train_X = X[:TRAINING_END]
train_Y = y[:TRAINING_END]
val_X = X[TRAINING_END:VALIDATIGN_END]
val_Y = y[TRAINING_END:VALIDATIGN_END]
test_X = X[VALIDATIGN_END:]
test_Y = y[VALIDATIGN_END:]

In [15]:
BATCH_SIZE = 64
train_set = tf.data.Dataset.from_tensor_slices((train_X, train_Y)).batch(BATCH_SIZE).shuffle(TRAINING_END).repeat()
val_set = tf.data.Dataset.from_tensor_slices((val_X, val_Y)).batch(BATCH_SIZE)
test_set = tf.data.Dataset.from_tensor_slices((test_X, test_Y)).batch(BATCH_SIZE)

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=8, input_shape=[14, 10], return_sequences=True),
    tf.keras.layers.LSTM(units=8, dropout=0.2),
    tf.keras.layers.Dense(units=1)
])

model.summary()
model.compile(loss='mae',
             optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
             metrics=['mae'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 14, 8)             608       
_________________________________________________________________
lstm_1 (LSTM)                (None, 8)                 544       
_________________________________________________________________
dense (Dense)                (None, 1)                 9         
Total params: 1,161
Trainable params: 1,161
Non-trainable params: 0
_________________________________________________________________


In [20]:
EPOCHS=300
history = model.fit(train_set,
                   epochs=EPOCHS,
                   steps_per_epoch=int(np.ceil(TRAINING_END / float(BATCH_SIZE))),
                   validation_data=val_set,
                   validation_steps=int(np.ceil((VALIDATIGN_END - TRAINING_END) / float(BATCH_SIZE))),
                   verbose=1)

Epoch 1/100
42/42 [==============================] - 1s 22ms/step - loss: 0.0287 - mae: 0.0289 - val_loss: 0.0279 - val_mae: 0.0279
Epoch 2/100
42/42 [==============================] - 1s 26ms/step - loss: 0.0292 - mae: 0.0295 - val_loss: 0.0285 - val_mae: 0.0286
Epoch 3/100
42/42 [==============================] - 1s 27ms/step - loss: 0.0287 - mae: 0.0290 - val_loss: 0.0280 - val_mae: 0.0279
Epoch 4/100
42/42 [==============================] - 1s 22ms/step - loss: 0.0293 - mae: 0.0296 - val_loss: 0.0282 - val_mae: 0.0281
Epoch 5/100
42/42 [==============================] - 1s 21ms/step - loss: 0.0294 - mae: 0.0297 - val_loss: 0.0300 - val_mae: 0.0301
Epoch 6/100
42/42 [==============================] - 1s 22ms/step - loss: 0.0296 - mae: 0.0299 - val_loss: 0.0286 - val_mae: 0.0286
Epoch 7/100
42/42 [==============================] - 1s 22ms/step - loss: 0.0289 - mae: 0.0291 - val_loss: 0.0279 - val_mae: 0.0278
Epoch 8/100
42/42 [==============================] - 1s 21ms/step - loss: 0.

42/42 [==============================] - 1s 25ms/step - loss: 0.0282 - mae: 0.0285 - val_loss: 0.0278 - val_mae: 0.0277
Epoch 63/100
42/42 [==============================] - 1s 22ms/step - loss: 0.0281 - mae: 0.0284 - val_loss: 0.0298 - val_mae: 0.0297
Epoch 64/100
42/42 [==============================] - 1s 21ms/step - loss: 0.0291 - mae: 0.0294 - val_loss: 0.0329 - val_mae: 0.0328
Epoch 65/100
42/42 [==============================] - 1s 23ms/step - loss: 0.0285 - mae: 0.0288 - val_loss: 0.0282 - val_mae: 0.0280
Epoch 66/100
42/42 [==============================] - 1s 26ms/step - loss: 0.0281 - mae: 0.0283 - val_loss: 0.0287 - val_mae: 0.0286
Epoch 67/100
42/42 [==============================] - 1s 23ms/step - loss: 0.0281 - mae: 0.0284 - val_loss: 0.0288 - val_mae: 0.0287
Epoch 68/100
42/42 [==============================] - 1s 26ms/step - loss: 0.0280 - mae: 0.0283 - val_loss: 0.0276 - val_mae: 0.0275
Epoch 69/100
42/42 [==============================] - 1s 27ms/step - loss: 0.0288 

In [21]:
model.evaluate(test_set)

13/13 [==============================] - 0s 10ms/step - loss: 0.0291 - mae: 0.0291


[0.02909910478270971, 0.029114084]